In [1]:
import pandas as pd  
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from sqlalchemy import create_engine, inspect, text
import pyodbc
from tqdm import tqdm
from typing import List
import sqlalchemy
from scipy.stats import norm
import time
import re
import joblib
import numpy as np
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from scipy.stats import mode
from scipy.stats import gaussian_kde
import statsmodels.api as sm
from sklearn.mixture import GaussianMixture
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier, 
                             ExtraTreesClassifier, AdaBoostClassifier)
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv("full_specnr_cleaned.csv")
df

,SPECNR,MANDT,SEQNR,DISKNR,SPECTYPE,CONMETHOD,TIK,TIKP,TIKM,DEFUALTTIK,...,INKTYPE,ROLLPITCH1_TOP,ROLLPITCH2_TOP,ROLLPITCH1_BOT,ROLLPITCH2_BOT,MAKTX_1,MAKTX_2,MAKTX_3,SPART,MACTIVE
0,21CSP002-04,300.0,1.0,1000050281.0,BUM-B,MSAP,0.250,0.020,0.020,NaN,...,NaN,0.00,0.0,0.00,0.0,NaN,NaN,NaN,30.0,X
1,21CSP002-05,300.0,1.0,1000050281.0,BUM-B,MSAP,0.250,0.020,0.020,NaN,...,NaN,0.00,0.0,0.00,0.0,NaN,NaN,NaN,30.0,NaN
2,21CSP002-06,300.0,1.0,1000050281.0,BUM-B,MSAP,0.250,0.020,0.020,NaN,...,NaN,0.00,0.0,0.00,0.0,NaN,NaN,NaN,30.0,NaN
3,21CSP002-07,300.0,2.0,1000050281.0,BUM-B,MSAP,0.250,0.020,0.020,NaN,...,NaN,0.00,0.0,0.00,0.0,NaN,NaN,NaN,30.0,NaN
4,21CSP002-08,300.0,1.0,1000050281.0,BUM-B,MSAP,0.250,0.020,0.020,NaN,...,NaN,0.32,0.0,0.32,0.0,NaN,NaN,NaN,30.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9164,25SIP002-01,300.0,7.0,1000057818.0,CORELESS,MSAP,0.134,0.020,0.020,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,X
9165,25SIP003-01,300.0,8.0,1000058085.0,CORELESS,TENTING,0.109,0.015,0.015,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,X
9166,25SIP003-02,300.0,1.0,1000058085.0,CORELESS,TENTING,0.109,0.015,0.015,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,X
9167,25SIP004-01,300.0,1.0,1000057928.0,BUM-B,TENTING,0.270,0.030,0.030,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,X


In [10]:
df1 = pd.read_csv("full_specnr_joined_1102.csv")
df1

,SPECNR,MANDT,SEQNR,DISKNR,SPECTYPE,CONMETHOD,TIK,TIKP,TIKM,DEFUALTTIK,...,FLEXCH,AUTOPUNCH,TARGETDSTXTOP,TARGETDSTXBOT,TARGETDSTYLEFT,TARGETDSTYRIGHT,MASKAR,MASKARC,EXCTIK,JIG_CHK
0,21CSP002-04,300,1,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0,0,0,0,0.0,0.0,40.0,0.0,0.0,NaN
1,21CSP002-05,300,2,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
2,21CSP002-06,300,1,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
3,21CSP002-07,300,2,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
4,21CSP002-08,300,2,1000050281,BUM-B,MSAP,0.25,0.02,0.02,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9160,25SIP015-02,300,3,1000058039,NORMAL,TENTING,0.25,0.04,0.04,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
9161,25SIP016-01,300,2,1000058047,NORMAL,TENTING,0.25,0.05,0.05,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
9162,25SIP017-01,300,2,1000058332,NORMAL,TENTING,0.25,0.04,0.04,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN
9163,25SIP018-01,300,2,1000058022,NORMAL,TENTING,0.25,0.04,0.04,NaN,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN


In [8]:
df['SPART'].value_counts()

SPART
30.0    9167
Name: count, dtype: int64

In [9]:
df['PRDHA_TXT'].value_counts()

PRDHA_TXT
PKG-CSP ; M-DRAM (NPOP)              1238
PKG-FCBOC ; DRAM                     1186
PKG-FCCSP ; Controller                855
PKG-CSP ; M-DRAM (POP)                847
PKG-CSP ; SSD                         753
FC-FCBGA ; All                        513
PKG-CSP ; Automotive                  334
PKG-CSP ; eMCP                        327
PKG-FCCSP ; Automotive                325
PKG-CSP ; Non Memory                  284
PKG-FCCSP ; UFS                       266
PKG-FCCSP ; Application Processor     221
PKG-SIP ; RF                          210
FC-FCBGA ; Automotive                 206
PKG-FCSIP ; RF                        196
PKG-CSP ; eMMC                        151
PKG-FCBOC ; TSV                       134
PKG-CSP ; DRAM                        132
PKG-FCCSP ; uMCP                      127
PKG-CSP ; UFS                         116
PKG-FCBGA ; All                       103
PKG-FCBOC ; GDDR                      101
PKG-FCCSP ; eMMC                       84
PKG-FCCSP ; Bitcoin     

In [11]:
df['PRDHA_TXT'].value_counts()

PRDHA_TXT
PKG-CSP ; M-DRAM (NPOP)              1238
PKG-FCBOC ; DRAM                     1186
PKG-FCCSP ; Controller                855
PKG-CSP ; M-DRAM (POP)                847
PKG-CSP ; SSD                         753
FC-FCBGA ; All                        513
PKG-CSP ; Automotive                  334
PKG-CSP ; eMCP                        327
PKG-FCCSP ; Automotive                325
PKG-CSP ; Non Memory                  284
PKG-FCCSP ; UFS                       266
PKG-FCCSP ; Application Processor     221
PKG-SIP ; RF                          210
FC-FCBGA ; Automotive                 206
PKG-FCSIP ; RF                        196
PKG-CSP ; eMMC                        151
PKG-FCBOC ; TSV                       134
PKG-CSP ; DRAM                        132
PKG-FCCSP ; uMCP                      127
PKG-CSP ; UFS                         116
PKG-FCBGA ; All                       103
PKG-FCBOC ; GDDR                      101
PKG-FCCSP ; eMMC                       84
PKG-FCCSP ; Bitcoin     

### 1102제거 버전

In [3]:
for col in df.columns:
    print(col)

SPECNR
MANDT
SEQNR
DISKNR
SPECTYPE
CONMETHOD
TIK
TIKP
TIKM
DEFUALTTIK
SMTIK
SMTIKP
SMTIKM
TIKRATE
DVPNO
DVPDATE
SYS_TOOL
SYS_KDMAT
SYS_REV
CAM_KDMAT
CAM_REV
OLD_TOOL
CAM_TOOL
HZ
HZ_CHK
MATNR_INFO
IMAGE_SCALE
REAL
TEST
EWONHWA
TIKBASE
TARTIK
TIKDIFF
BALLBOT
BALLTOP
UNITSIZEX
UNITSIZEY
UNITTOLM
UNITTOLP
UNITPITCH
UNITPITCHM
UNITPITCHP
UNITPITCHY
UNITPITCHMY
UNITPITCHPY
STRIPTOLM
STRIPTOLP
STRIPTOLM_Y
STRIPTOLP_Y
LPC
PKGCODE
SYMMETRY
KUNNRMATNR
SPLXOUTRATE
SPLXOUT
SPLXOPT
XOUTRATE
XOUT
XOPT
PURPOSE
MATNRGRADE
GRADEITEM
GRADECONTENT
ULNEED
ULPOS
ULPROC
ULFORMAT
WEEKNEED1
WEEKPOS1
WEEKPROC1
WEEKFORMAT1
WEEKNEED2
WEEKPOS2
WEEKPROC2
WEEKFORMAT2
PSRTENT
PLUGGING
WATERMK
DUALTOOL
EDMARK
FILTERTRA
PSRAPPR
TERM4
TWODBARCODE
UNIT2DID
HALOGENFREE
MEABM
GNSPECNR
UNITNO
STRIPNO
PRTYPE
MOLDGATE_ARRANGE
MOLDGATE_ARRTXT
STRIP_ARRANGE
STRIP_ARGTXT
BTBX
TOLERANCEXM
TOLERANCEXP
BTBY
TOLERANCEYM
TOLERANCEYP
STRCT
DETTYPE
LAMIN
LASER
M1
M2
M2_M1
B1
B2
B3
CAMYIELD
CONVERSION
SHIPPING
SSTRAY
NETPR
WAERK
YILD
S

### 처음버전

In [4]:
for col in df.columns:
    print(col)

SPECNR
MANDT
SEQNR
DISKNR
SPECTYPE
CONMETHOD
TIK
TIKP
TIKM
DEFUALTTIK
SMTIK
SMTIKP
SMTIKM
TIKRATE
DVPNO
DVPDATE
SYS_TOOL
SYS_KDMAT
SYS_REV
CAM_KDMAT
CAM_REV
OLD_TOOL
CAM_TOOL
HZ
HZ_CHK
MATNR_INFO
IMAGE_SCALE
REAL
TEST
EWONHWA
TIKBASE
TARTIK
TIKDIFF
BALLBOT
BALLTOP
UNITSIZEX
UNITSIZEY
UNITTOLM
UNITTOLP
UNITPITCH
UNITPITCHM
UNITPITCHP
UNITPITCHY
UNITPITCHMY
UNITPITCHPY
STRIPTOLM
STRIPTOLP
STRIPTOLM_Y
STRIPTOLP_Y
LPC
PKGCODE
SYMMETRY
KUNNRMATNR
SPLXOUTRATE
SPLXOUT
SPLXOPT
XOUTRATE
XOUT
XOPT
PURPOSE
MATNRGRADE
GRADEITEM
GRADECONTENT
ULNEED
ULPOS
ULPROC
ULFORMAT
WEEKNEED1
WEEKPOS1
WEEKPROC1
WEEKFORMAT1
WEEKNEED2
WEEKPOS2
WEEKPROC2
WEEKFORMAT2
PSRTENT
PLUGGING
WATERMK
DUALTOOL
EDMARK
FILTERTRA
PSRAPPR
TERM4
TWODBARCODE
UNIT2DID
HALOGENFREE
MEABM
GNSPECNR
UNITNO
STRIPNO
PRTYPE
MOLDGATE_ARRANGE
MOLDGATE_ARRTXT
STRIP_ARRANGE
STRIP_ARGTXT
BTBX
TOLERANCEXM
TOLERANCEXP
BTBY
TOLERANCEYM
TOLERANCEYP
STRCT
DETTYPE
LAMIN
LASER
M1
M2
M2_M1
B1
B2
B3
CAMYIELD
CONVERSION
SHIPPING
SSTRAY
NETPR
WAERK
YILD
S

### full_specnr_joined 확인

In [5]:
df1 = pd.read_csv("full_specnr_joined.csv")
df1.head()

,SPECNR,DUP_COUNT_ZTPP1110,DUP_COUNT_ZTPP1120,DUP_COUNT_ZTPP1130,DUP_COUNT_ZTPP1160,DUP_COUNT_ZTPP1170,DUP_COUNT_ZTPP1180,DUP_COUNT_ZTPP1190,DUP_COUNT_ZTPP1200,DUP_COUNT_ZTPP1370,...,TARGETDSTYLEFT,TARGETDSTYRIGHT,MASKAR,MASKARC,EXCTIK,JIG_CHK,ETL_DT.9,WERKS.9,DUP_COUNT_ZTPP1140.1,rn.9
0,21CAN002-01,1,50,2.0,20.0,4.0,30.0,1.0,73.0,77.0,...,0.0,0.0,0.0,0.0,0.0,NaN,2024-11-18 10:28:46.767,1110,66,1
1,21CAN003-02,2,76,4.0,16.0,4.0,44.0,2.0,48.0,98.0,...,0.0,0.0,0.0,0.0,0.0,NaN,2024-11-18 10:24:26.480,1110,122,1
2,21CAN005-01,1,16,3.0,3.0,2.0,6.0,1.0,3.0,13.0,...,0.0,0.0,0.0,0.0,0.0,NaN,2024-11-18 10:32:09.457,1110,84,1
3,21CAN008-01,1,45,2.0,8.0,2.0,28.0,1.0,49.0,46.0,...,0.0,0.0,0.0,0.0,0.0,NaN,2024-11-18 10:24:26.480,1110,61,1
4,21CAN008-02,1,45,2.0,8.0,2.0,28.0,1.0,49.0,46.0,...,0.0,0.0,0.0,0.0,0.0,NaN,2024-11-18 10:24:26.480,1110,61,1


In [19]:
df1['SAWTOSAW'].value_counts()

KeyError: 'SAWTOSAW'

In [6]:
for col in df1.columns:
    print(col)

SPECNR
DUP_COUNT_ZTPP1110
DUP_COUNT_ZTPP1120
DUP_COUNT_ZTPP1130
DUP_COUNT_ZTPP1160
DUP_COUNT_ZTPP1170
DUP_COUNT_ZTPP1180
DUP_COUNT_ZTPP1190
DUP_COUNT_ZTPP1200
DUP_COUNT_ZTPP1370
DUP_COUNT_ZTPP1140
MANDT
SPECNR.1
SEQNR
DISKNR
SPECTYPE
CONMETHOD
TIK
TIKP
TIKM
DEFUALTTIK
SMTIK
SMTIKP
SMTIKM
TIKRATE
DVPNO
DVPDATE
SYS_TOOL
SYS_KDMAT
SYS_REV
CAM_KDMAT
CAM_REV
OLD_TOOL
CAM_TOOL
HZ
HZ_CHK
MATNR_INFO
IMAGE_SCALE
REAL
TEST
EWONHWA
TIKBASE
TARTIK
TIKDIFF
BALLBOT
BALLTOP
UNITSIZEX
UNITSIZEY
UNITTOLM
UNITTOLP
UNITPITCH
UNITPITCHM
UNITPITCHP
UNITPITCHY
UNITPITCHMY
UNITPITCHPY
STRIPTOLM
STRIPTOLP
STRIPTOLM_Y
STRIPTOLP_Y
LPC
PKGCODE
SYMMETRY
KUNNRMATNR
SPLXOUTRATE
SPLXOUT
SPLXOPT
XOUTRATE
XOUT
XOPT
PURPOSE
MATNRGRADE
GRADEITEM
GRADECONTENT
ULNEED
ULPOS
ULPROC
ULFORMAT
WEEKNEED1
WEEKPOS1
WEEKPROC1
WEEKFORMAT1
WEEKNEED2
WEEKPOS2
WEEKPROC2
WEEKFORMAT2
PSRTENT
PLUGGING
WATERMK
DUALTOOL
EDMARK
FILTERTRA
PSRAPPR
TERM4
TWODBARCODE
UNIT2DID
HALOGENFREE
MEABM
GNSPECNR
UNITNO
STRIPNO
PRTYPE
MOLDGATE_ARRANGE
MOL

In [8]:

# 컬럼 이름 추출
cols1 = set(df.columns)
cols2 = set(df1.columns)

# 차이점 찾기
only_in_df = cols1 - cols2
only_in_df1 = cols2 - cols1
common_cols = cols1 & cols2

# 결과 출력
print("✅ 공통 컬럼:")
print(common_cols)

print("\n full_specnr_cleaned.csv에만 있는 컬럼:")
print(only_in_df)

print("\n full_specnr_joined.csv에만 있는 컬럼:")
print(only_in_df1)


✅ 공통 컬럼:
{'LASER', 'AENAM', 'TRIMSIZEY20', 'WIDMX', 'REF', 'BOGANG', 'SYS_KDMAT', 'LOWERLEFTXM', 'EDMARK', 'GAPCUPMIN', 'STRIPNO', 'STRIPTOLP_Y', 'PETCHBACK', 'LABST', 'CUDIRECT', 'PLATEYIELD', 'TARGETDSTYRIGHT', 'ETS', 'ULNEED', 'PINSTRC', 'PRLAYERDEP', 'MEINS', 'REAL', 'TOOLYEAR', 'OHM', 'ZAREA06', 'WATERMARK1', 'MOLDGATE_ARRTXT', 'LVH4PAD', 'OKDAT', 'ILEWIDMN', 'TRIMSIZEX20', 'PREWORK', 'DVPDATE', 'MANDT', 'UNITTOLM', 'FLAT_PAD', 'TRIMSIZEX02', 'ZSUM', 'STRIP_ARRANGE', 'MIDDLE1', 'LINEWID', 'STUBMIN', 'BPL', 'BALLBOT', 'CORELESS', 'TARGETDSTYLEFT', 'CUBEAM', 'SEPCSTAT', 'DPRLAYERDEP', 'TIKP', 'LVH3PAD', 'ERZET', 'ILEWID', 'TRIMSIZEX04', 'ETS_MTK', 'REVERSE', 'PROCGRNM', 'IVH4PAD', 'MATNRGRADE', 'STRIP_ARGTXT', 'CONMETHOD', 'AS_COMPLT', 'LINEWIDREF', 'COTINGBIT', 'XOPT', 'TOP2', 'MOLDGATE', 'SPEC9', 'PINMETH', 'PADSEQ', 'ZAUTO', 'GLICAP', 'ZAREA02', 'CUETSIZEMIN', 'SIGMETHOD', 'WATERMARK3', 'POSNR', 'LAYERPARDE', 'INTERVALY', 'PADTYPE', 'AOIGUBUN', 'HIPOT', 'CONTIK', 'ETA_MIN', 'ZDEP

In [12]:
# 컬럼 이름 추출
cols1 = set(df.columns)
cols2 = set(df1.columns)

# 차이점 찾기
only_in_df = cols1 - cols2
only_in_df1 = cols2 - cols1
common_cols = cols1 & cols2

# 컬럼 수 출력
print(f" full_specnr_cleaned 컬럼 수: {len(cols1)}")
print(f" full_specnr_joined (최근버전) 컬럼 수: {len(cols2)}")
print(f" 공통 컬럼 수: {len(common_cols)}")
print(f" full_specnr_cleaned에만 있는 컬럼 수: {len(only_in_df)}")
print(f" full_specnr_joined에만  있는 컬럼 수 (최근버전): {len(only_in_df1)}")

print("\n 공통 컬럼:")
print(common_cols)

print("\n full_specnr_cleaned에만 있는 컬럼:")
print(only_in_df)

print("\n full_specnr_joined에만 있는 컬럼 (최근버전):")
print(only_in_df1)

 full_specnr_cleaned 컬럼 수: 903
 full_specnr_joined (최근버전) 컬럼 수: 757
 공통 컬럼 수: 578
 full_specnr_cleaned에만 있는 컬럼 수: 325
 full_specnr_joined에만  있는 컬럼 수 (최근버전): 179

 공통 컬럼:
{'LASER', 'AENAM', 'TRIMSIZEY20', 'WIDMX', 'REF', 'BOGANG', 'SYS_KDMAT', 'LOWERLEFTXM', 'EDMARK', 'GAPCUPMIN', 'STRIPNO', 'STRIPTOLP_Y', 'PETCHBACK', 'LABST', 'CUDIRECT', 'PLATEYIELD', 'TARGETDSTYRIGHT', 'ETS', 'ULNEED', 'PINSTRC', 'PRLAYERDEP', 'MEINS', 'REAL', 'TOOLYEAR', 'OHM', 'ZAREA06', 'WATERMARK1', 'MOLDGATE_ARRTXT', 'LVH4PAD', 'OKDAT', 'ILEWIDMN', 'TRIMSIZEX20', 'PREWORK', 'DVPDATE', 'MANDT', 'UNITTOLM', 'FLAT_PAD', 'TRIMSIZEX02', 'ZSUM', 'STRIP_ARRANGE', 'MIDDLE1', 'LINEWID', 'STUBMIN', 'BPL', 'BALLBOT', 'CORELESS', 'TARGETDSTYLEFT', 'CUBEAM', 'SEPCSTAT', 'DPRLAYERDEP', 'TIKP', 'LVH3PAD', 'ERZET', 'ILEWID', 'TRIMSIZEX04', 'ETS_MTK', 'REVERSE', 'PROCGRNM', 'IVH4PAD', 'MATNRGRADE', 'STRIP_ARGTXT', 'CONMETHOD', 'AS_COMPLT', 'LINEWIDREF', 'COTINGBIT', 'XOPT', 'TOP2', 'MOLDGATE', 'SPEC9', 'PINMETH', 'PADSEQ', 'ZAUT

In [13]:
df2 = pd.read_csv("full_specnr_joined.csv")
df2.head()

,SPECNR,DUP_COUNT_ZTPP1110,DUP_COUNT_ZTPP1120,DUP_COUNT_ZTPP1130,DUP_COUNT_ZTPP1160,DUP_COUNT_ZTPP1170,DUP_COUNT_ZTPP1180,DUP_COUNT_ZTPP1190,DUP_COUNT_ZTPP1200,DUP_COUNT_ZTPP1370,...,TARGETDSTYLEFT,TARGETDSTYRIGHT,MASKAR,MASKARC,EXCTIK,JIG_CHK,ETL_DT.9,WERKS.9,DUP_COUNT_ZTPP1140.1,rn.9
0,21CAN002-01,1,50,2.0,20.0,4.0,30.0,1.0,73.0,77.0,...,0.0,0.0,0.0,0.0,0.0,NaN,2024-11-18 10:28:46.767,1110,66,1
1,21CAN003-02,2,76,4.0,16.0,4.0,44.0,2.0,48.0,98.0,...,0.0,0.0,0.0,0.0,0.0,NaN,2024-11-18 10:24:26.480,1110,122,1
2,21CAN005-01,1,16,3.0,3.0,2.0,6.0,1.0,3.0,13.0,...,0.0,0.0,0.0,0.0,0.0,NaN,2024-11-18 10:32:09.457,1110,84,1
3,21CAN008-01,1,45,2.0,8.0,2.0,28.0,1.0,49.0,46.0,...,0.0,0.0,0.0,0.0,0.0,NaN,2024-11-18 10:24:26.480,1110,61,1
4,21CAN008-02,1,45,2.0,8.0,2.0,28.0,1.0,49.0,46.0,...,0.0,0.0,0.0,0.0,0.0,NaN,2024-11-18 10:24:26.480,1110,61,1


### 수치형 / 범주형 컬럼 확인 

In [14]:
def classify_columns(df, unique_threshold=10, treat_numeric_low_card_as_categorical=False):
    """
    데이터프레임 컬럼을 수치형 / 범주형으로 자동 분류합니다.

    Parameters:
    - df: pandas DataFrame
    - unique_threshold: 고유값 수가 이 값 이하인 수치형 컬럼을 범주형으로 간주할지 결정하는 기준
    - treat_numeric_low_card_as_categorical: True로 설정 시, 고유값 수가 적은 수치형 컬럼도 범주형으로 처리

    Returns:
    - numerical_cols: 수치형 컬럼 리스트
    - categorical_cols: 범주형 컬럼 리스트
    """
    numerical_cols = []
    categorical_cols = []

    for col in df.columns:
        dtype = df[col].dtype

        if pd.api.types.is_numeric_dtype(dtype):
            if treat_numeric_low_card_as_categorical and df[col].nunique() <= unique_threshold:
                categorical_cols.append(col)
            else:
                numerical_cols.append(col)
        else:
            categorical_cols.append(col)
    
    return numerical_cols, categorical_cols

In [16]:
numerical_cols, categorical_cols = classify_columns(df)

print(" 수치형 컬럼:")
print(len(numerical_cols))
print(numerical_cols)

print("\n 범주형 컬럼:")
print(len(categorical_cols))
print(categorical_cols)

📊 수치형 컬럼:
662
['MANDT', 'SEQNR', 'TIK', 'TIKP', 'TIKM', 'SMTIK', 'SMTIKP', 'SMTIKM', 'TIKRATE', 'DVPDATE', 'EWONHWA', 'TARTIK', 'TIKDIFF', 'BALLBOT', 'BALLTOP', 'UNITSIZEX', 'UNITSIZEY', 'UNITTOLM', 'UNITTOLP', 'UNITPITCH', 'UNITPITCHM', 'UNITPITCHP', 'UNITPITCHY', 'UNITPITCHMY', 'UNITPITCHPY', 'STRIPTOLM', 'STRIPTOLP', 'STRIPTOLM_Y', 'STRIPTOLP_Y', 'LPC', 'SPLXOUTRATE', 'SPLXOUT', 'XOUTRATE', 'XOUT', 'PURPOSE', 'WEEKPOS2', 'WEEKPROC2', 'WEEKFORMAT2', 'WATERMK', 'DUALTOOL', 'EDMARK', 'FILTERTRA', 'TWODBARCODE', 'HALOGENFREE', 'MOLDGATE_ARRANGE', 'STRIP_ARRANGE', 'BTBX', 'TOLERANCEXM', 'TOLERANCEXP', 'BTBY', 'TOLERANCEYM', 'TOLERANCEYP', 'LAMIN', 'LASER', 'M2', 'M2_M1', 'B3', 'CAMYIELD', 'SSTRAY', 'NETPR', 'WAERK', 'YILD', 'SIZECHK', 'LOWERLEFTX', 'LOWERLEFTXP', 'LOWERLEFTXM', 'LOWERLEFTY', 'LOWERLEFTYP', 'LOWERLEFTYM', 'SOPBOT', 'ROTATE', 'LAYERNR', 'STRIPX', 'STRIPY', 'UPNR', 'UPCOL', 'UPROW', 'UPBLOCK', 'PANELMNLX', 'PANELMNLY', 'PANELLOSSX', 'PANELLOSSY', 'INTERVALX', 'INTERVALY', '

### AI 적용할 수 있는 컬럼 확인 필요

In [26]:
from typing import List, Dict
def determine_target_type(df: pd.DataFrame, target_columns: List[str], threshold_unique: int = 15) -> Dict[str, str]:
    result = {}
    for col in target_columns:
        if col not in df.columns:
            continue
        if pd.api.types.is_numeric_dtype(df[col]):
            n_unique = df[col].nunique(dropna=True)
            result[col] = "classification" if n_unique <= threshold_unique else "regression"
        else:
            result[col] = "classification"
    return result

# 제거 대상 컬럼 제외한 전체 컬럼을 타겟 후보로 설정
#y_candidates = [col for col in df.columns]

threshold_unique = 15
y_candidates = [
    col for col in df.columns
    if  df[col].nunique() <= threshold_unique
]

# 실제 존재하는 컬럼만 필터링 (이제 사실상 모든 컬럼이 여기에 해당)
y_list_exist = [y for y in y_candidates if y in df.columns]

# 분류/회귀 판별
target_types = determine_target_type(df, y_list_exist)

# 분류 타겟만 선택
classification_targets = [t for t, type_ in target_types.items() if type_ == "classification"]
print(len(classification_targets))
classification_targets

608


['MANDT',
 'SPECTYPE',
 'CONMETHOD',
 'DEFUALTTIK',
 'SMTIKP',
 'SMTIKM',
 'HZ',
 'HZ_CHK',
 'IMAGE_SCALE',
 'REAL',
 'TEST',
 'EWONHWA',
 'TIKBASE',
 'UNITTOLM',
 'UNITTOLP',
 'UNITPITCHM',
 'UNITPITCHP',
 'UNITPITCHMY',
 'UNITPITCHPY',
 'STRIPTOLM',
 'STRIPTOLP',
 'STRIPTOLM_Y',
 'STRIPTOLP_Y',
 'LPC',
 'SYMMETRY',
 'SPLXOPT',
 'XOPT',
 'PURPOSE',
 'MATNRGRADE',
 'ULNEED',
 'ULPOS',
 'ULPROC',
 'ULFORMAT',
 'WEEKNEED1',
 'WEEKPOS1',
 'WEEKPROC1',
 'WEEKFORMAT1',
 'WEEKNEED2',
 'WEEKPOS2',
 'WEEKPROC2',
 'WEEKFORMAT2',
 'PSRTENT',
 'PLUGGING',
 'WATERMK',
 'DUALTOOL',
 'EDMARK',
 'FILTERTRA',
 'PSRAPPR',
 'TERM4',
 'TWODBARCODE',
 'UNIT2DID',
 'HALOGENFREE',
 'MEABM',
 'PRTYPE',
 'MOLDGATE_ARRANGE',
 'MOLDGATE_ARRTXT',
 'STRIP_ARRANGE',
 'STRIP_ARGTXT',
 'STRCT',
 'DETTYPE',
 'LAMIN',
 'LASER',
 'M1',
 'M2',
 'M2_M1',
 'B1',
 'B2',
 'B3',
 'CAMYIELD',
 'SSTRAY',
 'NETPR',
 'WAERK',
 'YILD',
 'SIZECHK',
 'STICKY',
 'LOWERLEFTX',
 'LOWERLEFTXP',
 'LOWERLEFTXM',
 'LOWERLEFTY',
 'LOWERLEF

In [27]:
def determine_target_type(df: pd.DataFrame, target_columns: List[str], threshold_unique: int = 15) -> Dict[str, str]:
    result = {}
    for col in target_columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            n_unique = df[col].nunique(dropna=True)
            result[col] = "classification" if n_unique <= threshold_unique else "regression"
        else:
            result[col] = "classification"
    return result

# 고유값이 threshold_unique 이하인 컬럼을 타겟 후보로 설정
threshold_unique = 15
y_candidates = [col for col in df.columns if df[col].nunique() <= threshold_unique]

# 분류/회귀 판별
target_types = determine_target_type(df, y_candidates)

# 분류 타겟만 선택
classification_targets = [t for t, type_ in target_types.items() if type_ == "classification"]

print(f"🔍 분류 타겟 개수: {len(classification_targets)}")
classification_targets

🔍 분류 타겟 개수: 608


['MANDT',
 'SPECTYPE',
 'CONMETHOD',
 'DEFUALTTIK',
 'SMTIKP',
 'SMTIKM',
 'HZ',
 'HZ_CHK',
 'IMAGE_SCALE',
 'REAL',
 'TEST',
 'EWONHWA',
 'TIKBASE',
 'UNITTOLM',
 'UNITTOLP',
 'UNITPITCHM',
 'UNITPITCHP',
 'UNITPITCHMY',
 'UNITPITCHPY',
 'STRIPTOLM',
 'STRIPTOLP',
 'STRIPTOLM_Y',
 'STRIPTOLP_Y',
 'LPC',
 'SYMMETRY',
 'SPLXOPT',
 'XOPT',
 'PURPOSE',
 'MATNRGRADE',
 'ULNEED',
 'ULPOS',
 'ULPROC',
 'ULFORMAT',
 'WEEKNEED1',
 'WEEKPOS1',
 'WEEKPROC1',
 'WEEKFORMAT1',
 'WEEKNEED2',
 'WEEKPOS2',
 'WEEKPROC2',
 'WEEKFORMAT2',
 'PSRTENT',
 'PLUGGING',
 'WATERMK',
 'DUALTOOL',
 'EDMARK',
 'FILTERTRA',
 'PSRAPPR',
 'TERM4',
 'TWODBARCODE',
 'UNIT2DID',
 'HALOGENFREE',
 'MEABM',
 'PRTYPE',
 'MOLDGATE_ARRANGE',
 'MOLDGATE_ARRTXT',
 'STRIP_ARRANGE',
 'STRIP_ARGTXT',
 'STRCT',
 'DETTYPE',
 'LAMIN',
 'LASER',
 'M1',
 'M2',
 'M2_M1',
 'B1',
 'B2',
 'B3',
 'CAMYIELD',
 'SSTRAY',
 'NETPR',
 'WAERK',
 'YILD',
 'SIZECHK',
 'STICKY',
 'LOWERLEFTX',
 'LOWERLEFTXP',
 'LOWERLEFTXM',
 'LOWERLEFTY',
 'LOWERLEF

### 전체 범주 모델적용

In [28]:
columns_to_ignore = [col for col in df.columns if "DUP_COUNT" in col or re.search(r"\.\d+$", col)]

# %% [4] 전처리 함수 정의
def remove_rare_classes(df: pd.DataFrame, column: str, min_count: int = 10) -> pd.DataFrame:
    value_counts = df[column].value_counts()
    valid_values = value_counts[value_counts >= min_count].index
    return df[df[column].isin(valid_values)].reset_index(drop=True)

def get_feature_columns(df: pd.DataFrame, target_column: str, columns_to_ignore: List[str]) -> List[str]:
    exclude = {"SPECNR", target_column}
    return [col for col in df.columns if col not in exclude and col not in columns_to_ignore]

def preprocess_data(df: pd.DataFrame, feature_columns: List[str], target_column: str):
    for col in feature_columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna("missing")
        else:
            df[col] = df[col].fillna(0)

    df = df[feature_columns + [target_column]].dropna().reset_index(drop=True)

    X = df[feature_columns].copy()
    y = df[target_column].copy()

    feature_encoders = {}
    for col in X.columns:
        if X[col].dtype == 'object' or X[col].apply(type).nunique() > 1:
            X[col] = X[col].astype(str)
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
            feature_encoders[col] = le

    y_le = LabelEncoder()
    y = y_le.fit_transform(y)

    return X, y, y_le, feature_encoders

# %% [5] 모델 학습 함수
def train_models(X_train, X_test, y_train, y_test):
    models = {
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
        "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
        "LightGBM": LGBMClassifier(random_state=42, verbosity=-1),
        "RandomForest": RandomForestClassifier(random_state=42, n_jobs=-1)
    }

    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        report = classification_report(y_test, y_pred, output_dict=True)

        results[name] = {
            "model": model,
            "predictions": y_pred,
            "accuracy": report["accuracy"] * 100,
            "macro_f1": report["macro avg"]["f1-score"] * 100,
            "report": report
        }
    return results

def find_best_model(results):
    return max(results.items(), key=lambda x: x[1]["macro_f1"])

# %% [6] 전체 타겟 처리 & 최고 타겟 선택
all_results = []
for target in classification_targets:
    df_target = remove_rare_classes(df.copy(), target, min_count=10)
    feature_columns = get_feature_columns(df_target, target, columns_to_ignore)
    
    try:
        X, y, y_le, _ = preprocess_data(df_target, feature_columns, target)
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
        results = train_models(X_train, X_test, y_train, y_test)
        best_model_name, best_model_info = find_best_model(results)

        all_results.append({
            "target": target,
            "model_name": best_model_name,
            "model": best_model_info["model"],
            "macro_f1": best_model_info["macro_f1"],
            "report": best_model_info["report"],
            "y_test": y_test,
            "y_pred": best_model_info["predictions"],
            "label_encoder": y_le
        })
    except Exception as e:
        print(f"❌ {target}: 오류 발생 - {e}")

# %% [7] 최고 타겟 상세 분석 및 시각화
best_result = max(all_results, key=lambda x: x["macro_f1"])
target = best_result["target"]
y_test = best_result["y_test"]
y_pred = best_result["y_pred"]
y_le = best_result["label_encoder"]

print(f"\n🏆 최고 성능 타겟: {target} (모델: {best_result['model_name']}, Macro F1: {best_result['macro_f1']:.2f}%)")

# Classification Report
report_df = pd.DataFrame(best_result["report"]).transpose()
report_df = report_df.iloc[:-3]  # accuracy, macro avg, weighted avg 제외
print("\n📊 클래스별 성능:")
print(report_df[["precision", "recall", "f1-score", "support"]])


❌ MANDT: 오류 발생 - [09:01:34] C:\actions-runner\_work\xgboost\xgboost\src\objective\./regression_loss.h:68: Check failed: base_score > 0.0f && base_score < 1.0f: base_score must be in (0,1) for logistic loss, got: 0


  File "C:\nchoi\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\nchoi\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\nchoi\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                        pass_fds, cwd, env,
                        ^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
                        gid, gids, uid, umask,
                        ^^^^^^^^^^^^^^^^^^^^^^
                        start_new_session, process_group)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\nchoi\Lib\subprocess.py", line 1554, in _execute_child
    hp, ht, pid, tid = _winapi

❌ DEFUALTTIK: 오류 발생 - [09:05:10] C:\actions-runner\_work\xgboost\xgboost\src\objective\./regression_loss.h:68: Check failed: base_score > 0.0f && base_score < 1.0f: base_score must be in (0,1) for logistic loss, got: 0
❌ HZ_CHK: 오류 발생 - With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
❌ IMAGE_SCALE: 오류 발생 - With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.
❌ REAL: 오류 발생 - [09:09:41] C:\actions-runner\_work\xgboost\xgboost\src\objective\./regression_loss.h:68: Check failed: base_score > 0.0f && base_score < 1.0f: base_score must be in (0,1) for logistic loss, got: 0
❌ TEST: 오류 발생 - [09:09:43] C:\actions-runner\_work\xgboost\xgboost\src\objective\./regression_loss.h:68: Check failed: base_score > 0.0f && base_score < 1.0f: base_score must be in (0,1) for logistic loss, got: 0
❌ EWONHWA: 오류 발생 - With n_samples=0, tes

In [33]:
# ❗ 필요한 경우 정의
# classification_targets = [col for col in df.columns if col.startswith("CLASS_")]

# ❌ 무시할 컬럼 정의
columns_to_ignore = [col for col in df.columns if "DUP_COUNT" in col or re.search(r"\.\d+$", col)]

# ✅ 희소 클래스 제거
def remove_rare_classes(df: pd.DataFrame, column: str, min_count: int = 10) -> pd.DataFrame:
    value_counts = df[column].value_counts()
    valid_values = value_counts[value_counts >= min_count].index
    return df[df[column].isin(valid_values)].reset_index(drop=True)

def get_feature_columns(df: pd.DataFrame, target_column: str, columns_to_ignore: List[str]) -> List[str]:
    exclude = {"SPECNR", target_column}
    candidates = [col for col in df.columns if col not in exclude and col not in columns_to_ignore]

    # 상수형 변수(고정값 변수) 제외
    feature_columns = [col for col in candidates if df[col].nunique() > 1]

    return feature_columns

# ✅ 데이터 전처리
def preprocess_data(df: pd.DataFrame, feature_columns: List[str], target_column: str):
    for col in feature_columns:
        if df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].fillna("missing")
        else:
            df[col] = df[col].fillna(0)

    df = df[feature_columns + [target_column]].dropna().reset_index(drop=True)
    X = df[feature_columns].copy()
    y = df[target_column].copy()

    feature_encoders = {}
    for col in X.columns:
        if X[col].dtype == 'object' or X[col].apply(type).nunique() > 1:
            X[col] = X[col].astype(str)
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
            feature_encoders[col] = le

    y_le = LabelEncoder()
    y = y_le.fit_transform(y)

    return X, y, y_le, feature_encoders

# ✅ 모델 학습
def train_models(X_train, X_test, y_train, y_test):
    models = {
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42, base_score=0.5),
        #"CatBoost": CatBoostClassifier(verbose=0, random_state=42),
        #"LightGBM": LGBMClassifier(random_state=42, verbosity=-1),
        "RandomForest": RandomForestClassifier(random_state=42, n_jobs=-1)
    }

    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        report = classification_report(y_test, y_pred, output_dict=True)

        results[name] = {
            "model": model,
            "predictions": y_pred,
            "accuracy": report["accuracy"] * 100,
            "macro_f1": report["macro avg"]["f1-score"] * 100,
            "report": report
        }
    return results

# ✅ 최고 성능 모델 찾기
def find_best_model(results):
    return max(results.items(), key=lambda x: x[1]["macro_f1"])

# ✅ 전체 타겟별 모델링
all_results = []
for target in classification_targets:
    df_target = remove_rare_classes(df.copy(), target, min_count=10)

    if len(df_target) < 2:
        print(f"⚠️ {target}: 유효 샘플 수 부족 ({len(df_target)}개), 건너뜁니다.")
        continue

    feature_columns = get_feature_columns(df_target, target, columns_to_ignore)

    try:
        X, y, y_le, _ = preprocess_data(df_target, feature_columns, target)

        # stratify를 사용할 때 최소 클래스 수 확인
        unique_classes = pd.Series(y).nunique()
        stratify_param = y if unique_classes > 1 else None

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, stratify=y, test_size=0.2, random_state=42
        )

        results = train_models(X_train, X_test, y_train, y_test)
        best_model_name, best_model_info = find_best_model(results)

        all_results.append({
            "target": target,
            "model_name": best_model_name,
            "model": best_model_info["model"],
            "macro_f1": best_model_info["macro_f1"],
            "report": best_model_info["report"],
            "y_test": y_test,
            "y_pred": best_model_info["predictions"],
            "label_encoder": y_le
        })

    except Exception as e:
        print(f"❌ {target}: 오류 발생 - {e}")

# ✅ 최고 성능 모델 결과 출력
if all_results:
    best_result = max(all_results, key=lambda x: x["macro_f1"])
    target = best_result["target"]
    y_test = best_result["y_test"]
    y_pred = best_result["y_pred"]
    y_le = best_result["label_encoder"]

    print(f"\n🏆 최고 성능 타겟: {target} (모델: {best_result['model_name']}, Macro F1: {best_result['macro_f1']:.2f}%)")

    # 디코딩하여 리포트 보기
    y_test_decoded = y_le.inverse_transform(y_test)
    y_pred_decoded = y_le.inverse_transform(y_pred)

    # Classification Report
    report_df = pd.DataFrame(best_result["report"]).transpose().iloc[:-3]
    print("\n📊 클래스별 성능:")
    print(report_df[["precision", "recall", "f1-score", "support"]])
else:
    print("❗ 모델 학습 가능한 타겟이 없습니다.")

# ✅ 모델 학습에 성공한 타겟 변수 리스트 출력
successful_targets = [result["target"] for result in all_results]

print("\n✅ 모델 학습에 사용된 타겟 변수 목록:")
for i, var in enumerate(successful_targets, 1):
    print(f"{i}. {var}")

print(f"\n총 {len(successful_targets)}개 타겟 변수가 성공적으로 모델링되었습니다.")

⚠️ HZ_CHK: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ IMAGE_SCALE: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ EWONHWA: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ PURPOSE: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ ULNEED: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ ULPOS: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ ULPROC: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ ULFORMAT: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ WEEKNEED2: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ WEEKPOS2: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ WEEKPROC2: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ WEEKFORMAT2: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ PSRTENT: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ WATERMK: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ DUALTOOL: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ EDMARK: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ FILTERTRA: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ PSRAPPR: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ TWODBARCODE: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ HALOGENFREE: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ M2: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ M2_M1: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ B3: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ SSTRAY: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ WAERK: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ SIZECHK: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ PRECOATTOP: 유효 샘플 수 부족 (0개), 건너뜁니다.
⚠️ SOPBOT: 유효 샘플 수 부족

In [ ]:
# ❌ 무시할 컬럼 정의
columns_to_ignore = [col for col in df.columns if "DUP_COUNT" in col or re.search(r"\.\d+$", col)]

# ✅ 희소 클래스 제거
def remove_rare_classes(df: pd.DataFrame, column: str, min_count: int = 10) -> pd.DataFrame:
    value_counts = df[column].value_counts()
    valid_values = value_counts[value_counts >= min_count].index
    return df[df[column].isin(valid_values)].reset_index(drop=True)

# ✅ 상수형 컬럼 제거 포함
def get_feature_columns(df: pd.DataFrame, target_column: str, columns_to_ignore: List[str]) -> List[str]:
    exclude = {"SPECNR", target_column}
    candidates = [col for col in df.columns if col not in exclude and col not in columns_to_ignore]
    return [col for col in candidates if df[col].nunique(dropna=False) > 1]

# ✅ 데이터 전처리 (모든 입력을 문자열로 변환 → LabelEncoder 오류 방지)
def clean_column_values(series):
    """컬럼 값들을 안전하게 정리 (방법 2)"""
    # 1. 특수 값들을 먼저 처리
    series = series.replace([np.inf, -np.inf], np.nan)
    
    # 2. 모든 값을 문자열로 변환
    series = series.fillna('missing').astype(str)
    
    # 3. 문자열로 변환된 nan 값들 처리
    series = series.replace(['nan', 'None', '<NA>', 'NaN'], 'missing')
    
    return series

def preprocess_data(df: pd.DataFrame, feature_columns: List[str], target_column: str):
    """개선된 데이터 전처리 함수 (3가지 방법 통합)"""
    
    # 기본 전처리
    for col in feature_columns:
        if df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].fillna("missing")
        else:
            df[col] = df[col].fillna(0)

    df = df[feature_columns + [target_column]].dropna().reset_index(drop=True)
    X = df[feature_columns].copy()
    y = df[target_column].copy()

    feature_encoders = {}
    
    for col in X.columns:
        try:
            # 🔍 방법 3: 타입 확인 후 처리
            type_counts = X[col].apply(type).value_counts()
            print(f"📊 {col} 컬럼 타입 분포: {dict(type_counts)}")
            
            # 🔧 방법 2: 특수 값 사전 처리
            X[col] = clean_column_values(X[col])
            
            # 🔧 방법 1: 강제 타입 변환 강화
            X[col] = X[col].astype(str).replace('nan', 'missing').replace('None', 'missing')
            
            # 최종 검증: 모든 값이 문자열인지 확인
            final_types = X[col].apply(type).unique()
            if len(final_types) > 1:
                print(f"⚠️ {col}: 여전히 타입이 혼재되어 있음 - {final_types}")
                # 추가 강제 변환
                X[col] = pd.Series(X[col]).astype(str)
            
            # LabelEncoder 적용
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
            feature_encoders[col] = le
            
            print(f"✅ {col}: LabelEncoder 성공")

        except Exception as e:
            print(f"❌ LabelEncoder 변환 실패 - {col}: {e}")
            
            # 🔍 디버깅을 위한 추가 정보
            print(f"   - 최종 데이터 타입들: {X[col].apply(type).unique()}")
            print(f"   - 샘플 값들: {X[col].head().tolist()}")
            print(f"   - 고유값 개수: {X[col].nunique()}")
            
            # 컬럼 제거
            X.drop(columns=[col], inplace=True)

    # 타겟 변수 처리
    y_le = LabelEncoder()
    y = y_le.fit_transform(y)

    return X, y, y_le, feature_encoders

# ✅ 모델 학습
def train_models(X_train, X_test, y_train, y_test):
    models = {
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42, base_score=0.5),
        "RandomForest": RandomForestClassifier(random_state=42, n_jobs=-1)
    }

    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        report = classification_report(y_test, y_pred, output_dict=True)
        results[name] = {
            "model": model,
            "predictions": y_pred,
            "accuracy": report["accuracy"] * 100,
            "macro_f1": report["macro avg"]["f1-score"] * 100,
            "report": report
        }
    return results

# ✅ 최고 성능 모델 찾기
def find_best_model(results):
    return max(results.items(), key=lambda x: x[1]["macro_f1"])

# ✅ 전체 타겟별 모델링
all_results = []
for target in categorical_cols:
    df_target = remove_rare_classes(df.copy(), target, min_count=10)

    # 클래스 수 부족 시 건너뜀
    unique_classes = df_target[target].nunique()
    if unique_classes < 2:
        print(f"⚠️ {target}: 클래스 수 부족 ({unique_classes}개), 건너뜁니다.")
        continue

    feature_columns = get_feature_columns(df_target, target, columns_to_ignore)
    try:
        X, y, y_le, _ = preprocess_data(df_target, feature_columns, target)

        stratify_param = y if pd.Series(y).nunique() > 1 else None
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, stratify=stratify_param, test_size=0.2, random_state=42
        )

        results = train_models(X_train, X_test, y_train, y_test)
        best_model_name, best_model_info = find_best_model(results)

        all_results.append({
            "target": target,
            "model_name": best_model_name,
            "model": best_model_info["model"],
            "macro_f1": best_model_info["macro_f1"],
            "report": best_model_info["report"],
            "y_test": y_test,
            "y_pred": best_model_info["predictions"],
            "label_encoder": y_le
        })

    except Exception as e:
        print(f"❌ {target}: 오류 발생 - {e}")

# ✅ 최고 성능 모델 결과 출력
if all_results:
    best_result = max(all_results, key=lambda x: x["macro_f1"])
    target = best_result["target"]
    y_test = best_result["y_test"]
    y_pred = best_result["y_pred"]
    y_le = best_result["label_encoder"]

    print(f"\n🏆 최고 성능 타겟: {target} (모델: {best_result['model_name']}, Macro F1: {best_result['macro_f1']:.2f}%)")

    y_test_decoded = y_le.inverse_transform(y_test)
    y_pred_decoded = y_le.inverse_transform(y_pred)

    report_df = pd.DataFrame(best_result["report"]).transpose().iloc[:-3]
    print("\n📊 클래스별 성능:")
    print(report_df[["precision", "recall", "f1-score", "support"]])
else:
    print("❗ 모델 학습 가능한 타겟이 없습니다.")

# ✅ 모델 학습에 성공한 타겟 변수 리스트 출력
successful_targets = [result["target"] for result in all_results]
print("\n✅ 모델 학습에 사용된 타겟 변수 목록:")
for i, var in enumerate(successful_targets, 1):
    print(f"{i}. {var}")
print(f"\n총 {len(successful_targets)}개 타겟 변수가 성공적으로 모델링되었습니다.")

⚠️ SPECNR: 클래스 수 부족 (0개), 건너뜁니다.
❌ LabelEncoder 변환 실패 - SEQNR: 2767.0
   - 최종 데이터 타입들: [<class 'float'>]
   - 샘플 값들: [1.0, 1.0, 1.0, 2.0, 1.0]
   - 고유값 개수: 10
❌ LabelEncoder 변환 실패 - SPECTYPE: "SPECTYPE\n<class 'str'>    2767\nName: count, dtype: int64"
   - 최종 데이터 타입들: [<class 'str'>]
   - 샘플 값들: ['BUM-B', 'BUM-B', 'BUM-B', 'BUM-B', 'BUM-B']
   - 고유값 개수: 6
❌ LabelEncoder 변환 실패 - CONMETHOD: "CONMETHOD\n<class 'str'>    2767\nName: count, dtype: int64"
   - 최종 데이터 타입들: [<class 'str'>]
   - 샘플 값들: ['MSAP', 'MSAP', 'MSAP', 'MSAP', 'MSAP']
   - 고유값 개수: 4
❌ LabelEncoder 변환 실패 - TIK: 2767.0
   - 최종 데이터 타입들: [<class 'float'>]
   - 샘플 값들: [0.25, 0.25, 0.25, 0.25, 0.25]
   - 고유값 개수: 49
❌ LabelEncoder 변환 실패 - TIKP: 2767.0
   - 최종 데이터 타입들: [<class 'float'>]
   - 샘플 값들: [0.02, 0.02, 0.02, 0.02, 0.02]
   - 고유값 개수: 11
❌ LabelEncoder 변환 실패 - TIKM: 2767.0
   - 최종 데이터 타입들: [<class 'float'>]
   - 샘플 값들: [0.02, 0.02, 0.02, 0.02, 0.02]
   - 고유값 개수: 11
❌ LabelEncoder 변환 실패 - SMTIK: 2767.0
   - 최종 데이터 타입들: [

### CatBoost

In [48]:
# ❌ 무시할 컬럼 정의
def get_columns_to_ignore(df: pd.DataFrame) -> List[str]:
    return [col for col in df.columns if "DUP_COUNT" in col or re.search(r"\.\d+$", col)]

# ✅ 희소 클래스 제거
def remove_rare_classes(df: pd.DataFrame, column: str, min_count: int = 10) -> pd.DataFrame:
    value_counts = df[column].value_counts()
    valid_values = value_counts[value_counts >= min_count].index
    return df[df[column].isin(valid_values)].reset_index(drop=True)

# ✅ 상수형 컬럼 제거 포함
def get_feature_columns(df: pd.DataFrame, target_column: str, columns_to_ignore: List[str]) -> List[str]:
    exclude = {"SPECNR", target_column}
    candidates = [col for col in df.columns if col not in exclude and col not in columns_to_ignore]
    return [col for col in candidates if df[col].nunique(dropna=False) > 1]

# ✅ CatBoost용 데이터 전처리
def preprocess_data_for_catboost(df: pd.DataFrame, feature_columns: List[str], target_column: str):
    df = df[feature_columns + [target_column]].copy()

    for col in feature_columns:
        if df[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].fillna("missing").astype(str)
        else:
            df[col] = df[col].fillna(0)

    df[target_column] = df[target_column].fillna("missing").astype(str)

    X = df[feature_columns]
    y = df[target_column]

    cat_features = [col for col in feature_columns if X[col].dtype == 'object' or X[col].nunique() < 100]

    return X, y, cat_features

# ✅ CatBoost 모델 학습
def train_catboost_model(X_train, X_test, y_train, y_test, cat_features: List[str]):
    model = CatBoostClassifier(verbose=0, random_state=42)
    model.fit(X_train, y_train, cat_features=cat_features)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    return {
        "model": model,
        "predictions": y_pred,
        "accuracy": report["accuracy"] * 100,
        "macro_f1": report["macro avg"]["f1-score"] * 100,
        "report": report
    }

# ✅ 전체 타겟별 모델링
def run_catboost_pipeline(df: pd.DataFrame, categorical_cols: List[str]):
    columns_to_ignore = get_columns_to_ignore(df)
    all_results = []

    for target in categorical_cols:
        df_target = remove_rare_classes(df.copy(), target, min_count=10)

        unique_classes = df_target[target].nunique()
        if unique_classes < 2:
            print(f"⚠️ {target}: 클래스 수 부족 ({unique_classes}개), 건너뜁니다.")
            continue

        feature_columns = get_feature_columns(df_target, target, columns_to_ignore)

        try:
            X, y, cat_features = preprocess_data_for_catboost(df_target, feature_columns, target)
            stratify_param = y if y.nunique() > 1 else None

            X_train, X_test, y_train, y_test = train_test_split(
                X, y, stratify=stratify_param, test_size=0.2, random_state=42
            )

            result = train_catboost_model(X_train, X_test, y_train, y_test, cat_features)

            all_results.append({
                "target": target,
                "model": result["model"],
                "macro_f1": result["macro_f1"],
                "report": result["report"],
                "y_test": y_test,
                "y_pred": result["predictions"]
            })

        except Exception as e:
            print(f"❌ {target}: 오류 발생 - {e}")

    return all_results

# ✅ 최고 성능 모델 출력
def print_best_model_result(all_results: List[Dict]):
    if not all_results:
        print("❗ 모델 학습 가능한 타겟이 없습니다.")
        return

    best_result = max(all_results, key=lambda x: x["macro_f1"])
    target = best_result["target"]
    y_test = best_result["y_test"]
    y_pred = best_result["y_pred"]

    print(f"\n🏆 최고 성능 타겟: {target} (Macro F1: {best_result['macro_f1']:.2f}%)")

    report_df = pd.DataFrame(best_result["report"]).transpose().iloc[:-3]
    print("\n📊 클래스별 성능:")
    print(report_df[["precision", "recall", "f1-score", "support"]])

    successful_targets = [result["target"] for result in all_results]
    print("\n✅ 모델 학습에 사용된 타겟 변수 목록:")
    for i, var in enumerate(successful_targets, 1):
        print(f"{i}. {var}")
    print(f"\n총 {len(successful_targets)}개 타겟 변수가 성공적으로 모델링되었습니다.")

In [19]:
df['NETPR'].value_counts()

NETPR
0.0    9169
Name: count, dtype: int64